<a href="https://colab.research.google.com/github/soohyunme/TensorFlow_Tutorial/blob/main/Code/03_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [404]:
import os
from threading import active_count

from tensorflow.python.keras.layers.core import Activation
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10


# Device setting

In [405]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

# Load data

In [406]:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

# Normalize

In [407]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Initialize model

In [408]:
model = keras.Sequential(
    [
     keras.Input(shape=(32,32,3)),
     layers.Conv2D(32, 3, padding='valid', activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(64, 3, activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(128, 3, activation='relu'),
     layers.Flatten(),
     layers.Dense(64,activation='relu'),
     layers.Dense(10),

    ]
)

In [409]:
print(model.summary())

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_219 (Conv2D)          (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_100 (MaxPoolin (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_220 (Conv2D)          (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_101 (MaxPoolin (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_221 (Conv2D)          (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten_69 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_141 (Dense)            (None, 64)              

In [410]:
def my_model():
  with tf.device('/device:GPU:0'):
      inputs = keras.Input(shape=(32, 32, 3))
      x = layers.Conv2D(32, 3)(inputs)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      
      x = layers.Conv2D(64, 3)(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      
      x = layers.Conv2D(128, 3)(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      
      x = layers.Flatten()(x)
      x = layers.Dense(64, activation="relu")(x)
      outputs = layers.Dense(10)(x)
      model = keras.Model(inputs=inputs, outputs=outputs)
      return model

## New model


https://davinci-ai.tistory.com/29
 
VGGNet스타일의 14 Layers 네트워크 모델 참고

In [411]:
def new_model():
   with tf.device('/device:GPU:0'):
      inputs = keras.Input(shape=(32,32,3))
      x = layers.Conv2D(32,3)(inputs)
      x = layers.Conv2D(64, 3, padding='same',activation='relu')(x)
      x = layers.BatchNormalization()(x)
      x = layers.MaxPooling2D()(x)
      
      x = layers.Conv2D(128, 3, padding='same',activation='relu')(x)
      x = layers.Conv2D(256, 3, padding='valid',activation='relu')(x)
      x = layers.BatchNormalization()(x)

      x = layers.Flatten()(x)
      x = layers.Dense(128, activation='relu')(x)
      x = layers.Dense(64, activation='relu')(x)
      outputs = layers.Dense(10)(x)
      model = keras.Model(inputs=inputs, outputs=outputs)
      return model


In [412]:
# model = my_model()
model = new_model()

In [413]:
print(model.summary())


Model: "model_47"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_72 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_222 (Conv2D)          (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_223 (Conv2D)          (None, 30, 30, 64)        18496     
_________________________________________________________________
batch_normalization_152 (Bat (None, 30, 30, 64)        256       
_________________________________________________________________
max_pooling2d_102 (MaxPoolin (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_224 (Conv2D)          (None, 15, 15, 128)       73856     
_________________________________________________________________
conv2d_225 (Conv2D)          (None, 13, 13, 256)       295

# Model compile

In [414]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(lr=3e-4),
    metrics=['accuracy'],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


# Train and evaluate

In [402]:
with tf.device('/device:GPU:0'):
  history = model.fit(x_train, y_train, batch_size=32, epochs=10, verbose=2)
  model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/10
1563/1563 - 11s - loss: 1.2351 - accuracy: 0.5710
Epoch 2/10
1563/1563 - 10s - loss: 0.7521 - accuracy: 0.7400
Epoch 3/10
1563/1563 - 10s - loss: 0.4772 - accuracy: 0.8349
Epoch 4/10
1563/1563 - 10s - loss: 0.2631 - accuracy: 0.9102
Epoch 5/10
1563/1563 - 10s - loss: 0.1473 - accuracy: 0.9497
Epoch 6/10
1563/1563 - 10s - loss: 0.1095 - accuracy: 0.9643
Epoch 7/10
1563/1563 - 10s - loss: 0.0847 - accuracy: 0.9713
Epoch 8/10
1563/1563 - 10s - loss: 0.0768 - accuracy: 0.9744
Epoch 9/10
1563/1563 - 10s - loss: 0.0618 - accuracy: 0.9797
Epoch 10/10
1563/1563 - 10s - loss: 0.0616 - accuracy: 0.9799
313/313 - 1s - loss: 1.4739 - accuracy: 0.7407


# SUGGESTIONS


1. **What accuracy can you get on the test set by training longer, increasing the model size, changing kernel sizes, etc?**
> Baseline : 0.7197  
> Add MaxPooling each Conv2D : 0.6812  
> More Conv2D layer : 0.7133  
> Change kernel size : 0.6547  
> Change kernel size +  more epochs : 0.6669  
> Increase batch size + more epochs : 0.7007  
> Increase batch size : 0.6738  
> Change model architecture + reduce batch size : 0.7407

2. **In the last video we train a FC on MNIST: what can you get by using a conv net instead?**
> FC보다 파라미터의 수를 줄일 수 있음  
> 이미지의 형상을 고려하여 학습할 수 있음  
> 보다 적은 양의 학습 데이터로도 더 빠른 시간으로 학습이 가능